In [139]:
#Load the penguin's dataset using the following code.


import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the penguins dataset
df = sns.load_dataset("penguins")

df.dropna(inplace=True)

# Filter rows for 'Adelie' and 'Chinstrap' classes
selected_classes = ['Adelie', 'Chinstrap']
df_filtered = df[df['species'].isin(selected_classes)].copy()  # Make a copy to avoid the warning

# Initialize the LabelEncoder
le = LabelEncoder()

# Encode the species column
y_encoded = le.fit_transform(df_filtered['species'])

df_filtered['class_encoded'] = y_encoded

# Display the filtered and encoded DataFrame
print(df_filtered[['species', 'class_encoded']])

# Split the data into features (X) and target variable (y)

y = df_filtered['class_encoded']  # Target variable
X = df_filtered.drop(['species', 'island', 'sex','class_encoded'], axis=1)

       species  class_encoded
0       Adelie              0
1       Adelie              0
2       Adelie              0
4       Adelie              0
5       Adelie              0
..         ...            ...
215  Chinstrap              1
216  Chinstrap              1
217  Chinstrap              1
218  Chinstrap              1
219  Chinstrap              1

[214 rows x 2 columns]


1) What is the purpose of "y_encoded = le.fit_transform(df_filtered['species'])" ?
To encode the species column into numeric values. This is necessary because the LogisticRegression model can only work with numeric data. 0 for Adelie and 1 for Chinstrap

2) What is the purpose of "X = df.drop(['species', 'island', 'sex'], axis=1)" ?
To create a new DataFrame that only contains the features that we want to use for the model.

3) Why we cannot use "island" and "sex" features?
They are not relevant to the classification of penguins into Adelie and Chinstrap.

In [140]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape , X_test.shape , y_train.shape , y_test.shape)

(171, 4) (43, 4) (171,) (43,)


In [141]:
#Train the logistic regression model. Here we are using saga solver to learn weights.
logreg = LogisticRegression(solver='saga')

logreg.fit(X_train, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(logreg.coef_, logreg.intercept_)

Accuracy: 0.5813953488372093
[[ 2.76035852e-03 -8.24299288e-05  4.72001671e-04 -2.87065965e-04]] [-8.44846925e-06]


c:\Users\uzuma\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


1) Why is accuracy low? why does the saga solver perform poorly?

Since, in this case, we used 171 training datasets, the poor performance of the saga solver is attributed to the limited number of datasets.

In [142]:
#Change the solver to "liblinear"

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)
# Predict on the testing data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(logreg.coef_, logreg.intercept_)

Accuracy: 1.0
[[ 1.59665154 -1.42501103 -0.15238046 -0.003951  ]] [-0.0755452]


1) Why is accuracy now? why does the "liblinear" solver perform better than "saga" solver ? 

 The performance of the "saga" solver can be affected by the size of the dataset. For small datasets, "liblinear" perform better due to its simplicity, while "saga" require more data to perform well.

In [143]:
#Repeat the above tasks after feature normalization and observe the accuracy levels.

from sklearn.preprocessing import StandardScaler
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression(solver='saga')
logreg.fit(X_train_scaled, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy (saga):", accuracy)

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_scaled, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy (liblinear):", accuracy)



Accuracy (saga): 0.9767441860465116
Accuracy (liblinear): 0.9767441860465116


1) Now observe the accuracies for both  "liblinear" solver and "saga" solver. Why accuracy of the "saga" solver is increased?

Logistic regression is sensitive to the scale of features. Standard scaling can help logistic regression models, including the "saga" solver, perform better by ensuring that features are on similar scales. This scaling can lead to more stable and faster convergence.

In [144]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Load the penguins dataset
df = sns.load_dataset("penguins")

df.dropna(inplace=True)

# Filter rows for 'Adelie' and 'Chinstrap' classes
selected_classes = ['Adelie', 'Chinstrap']
df_filtered = df[df['species'].isin(selected_classes)].copy()  # Make a copy to avoid the warning

# Initialize the LabelEncoder
le = LabelEncoder()

# Encode the species column
y_encoded = le.fit_transform(df_filtered['species'])
df_filtered['class_encoded'] = y_encoded


df_filtered.head()

X = df_filtered.drop(['species', 'class_encoded'], axis=1)  # Choose features
y = df_filtered['class_encoded']  # Target variable

X.head()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression(solver='saga')

#logreg = LogisticRegression(max_iter=166, solver='newton-cg')
# logreg = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=100, multi_class='ovr', random_state=42)
logreg.fit(X_train, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(logreg.coef_, logreg.intercept_)

ValueError: could not convert string to float: 'Dream'

What is the problem? Why algorithm cannot perform classification?
How to solve this issue?

‘Dream’ cannot be converted to a float. For logistic regression, only numerical values can be applied. In this case, there are some strings, so to solve this problem, we need to convert those string features into numerical values accordingly.

In [145]:
#Use the following code, to replace string features with numerical ones.
df_filtered = pd.get_dummies(df_filtered, columns=['island', 'sex'], drop_first=True)
df_filtered.head()

#Use the following code to visualize the encoding

samples = df_filtered.groupby('sex_Male').head(1)
print(samples)
print()
samples = df_filtered.groupby('island_Torgersen').head(1)
print(samples)
print()
samples = df_filtered.groupby('island_Dream').head(1)
print(samples)


#Use the following code to apply logistic regression
X = df_filtered.drop(['species','class_encoded'], axis=1)

y = df_filtered['class_encoded']  # Target variable
print(X.shape, y.shape)
X.head()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.preprocessing import MaxAbsScaler
scaler=MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression(solver='saga',max_iter=150,)

#logreg = LogisticRegression(max_iter=166, solver='newton-cg')
# logreg = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=100, multi_class='ovr', random_state=42)
logreg.fit(X_train_scaled, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(logreg.coef_, logreg.intercept_)

  species  bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  \
0  Adelie            39.1           18.7              181.0       3750.0   
1  Adelie            39.5           17.4              186.0       3800.0   

   class_encoded  island_Dream  island_Torgersen  sex_Male  
0              0         False              True      True  
1              0         False              True     False  

   species  bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  \
0   Adelie            39.1           18.7              181.0       3750.0   
20  Adelie            37.8           18.3              174.0       3400.0   

    class_encoded  island_Dream  island_Torgersen  sex_Male  
0               0         False              True      True  
20              0         False             False     False  

   species  bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  \
0   Adelie            39.1           18.7              181.0       3750.0   
30  Adelie    

Why we are using the "MaxAbsScaler" scaler rather than the "StandardScaler"?

Data doesn't have a Gaussian distribution, "MaxAbsScaler" can preserve the original scale of features.

In [159]:
#Use the following code to visualize feature scaling before and after normalization.
from sklearn.preprocessing import MaxAbsScaler
scaler=MaxAbsScaler()
print(X_test)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled)

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled)

     bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  \
14             34.6           21.1              198.0       4400.0   
203            51.4           19.0              201.0       3950.0   
72             39.6           17.2              196.0       3550.0   
197            50.8           18.5              201.0       4450.0   
123            41.4           18.5              202.0       3875.0   
117            37.3           20.5              199.0       3775.0   
20             37.8           18.3              174.0       3400.0   
92             34.0           17.1              185.0       3400.0   
81             42.9           17.6              196.0       4700.0   
150            36.0           17.1              187.0       3700.0   
188            47.6           18.3              195.0       3850.0   
176            46.7           17.9              195.0       3300.0   
147            36.6           18.4              184.0       3475.0   
79             42.1 

What can you observe in the values related to "island_Dream",    "island_Torgersen"  and   "sex_Male" features before and after scaling?

Before scaling they have true or false values, after scaling they have numerical values.